In [1]:
import time
import numpy as np
import cv2
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
import os, errno
import glob
from scipy import stats

In [2]:
def VideoPull(back_frame, current_frame, file_name, path):
    video = [(os.path.join(root, file), file[0:-4]) 
        for root, dirs, files in os.walk(path) 
        for file in files 
        if file.endswith(file_name + '.mp4')]
    cam = cv2.VideoCapture(video[0][0])
    cam.set(1,back_frame)
    ret, f = cam.read(1)
    background = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
    cam.set(1,current_frame)
    ret, c = cam.read(1)
    current = cv2.cvtColor(c, cv2.COLOR_BGR2GRAY)
    difference = cv2.subtract(current,background)

    return background, current, difference, c

In [3]:
def Frame(name, df):
    start, end = df[df.name.str.contains(name)].start_frame.values, df[df.name.str.contains(name)].end_frame.values
    end = [int(elem) for elem in end]
    return(start,end)

In [11]:
video_list = r"E:\Downloads\ManducaMultiSenseData\Videos"
window_path = r"E:\Downloads\ManducaMultiSenseData\v3\Centroids_Filtered_v3\window\filteredFrameWindow.csv"
centroids = r"E:\Downloads\ManducaMultiSenseData\v3\Centroids_Filtered_v3"

In [28]:
names = np.asarray([('\\L0.1_c-3_m32_16_149643.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m32_17_154000.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m34_2_29617.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m34_11_45114.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m34_14_51383.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m34_22_74562.csv', 'already filtered out'),
 ('\\L0.1_c-3_m34_25_81816.csv', 'already filtered out'),
('\\L0.1_c-3_m38_4_42921.csv',  'removed since moth and tracking were working well'),
('\\L0.1_c-3_m40_9_31581.csv',  'removed since moth and tracking were working well'),
('\\L0.1_c-3_m40_15_46420.csv', 'removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames'),
('\\L0.1_c-3_m40_17_65761.csv', 'removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames'),
 ('\\L0.1_c-3_m41_5_58180.csv',  'removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames'),
('\\L0.1_c-3_m41_9_71991.csv',  'removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames'),
('\\L0.1_c-3_m41_11_76152.csv', 'removed since moth and tracking were working well. Few frames in the beginning and end snap between moth and flower. Might removed first 100 and last 100 frames'),
 ('\\L0.1_c-3_m45_5_40142.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m46_13_47364.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m47_4_22585.csv', 'abnormal moth behavior'),
 ('\\L0.1_c-3_m5_0_26767.csv',   'moth sits on flower'),
 ('\\L0.1_c-3_m5_3_36270.csv',  'moth sits on flower'),
 ('\\L0.1_c-3_m5_4_41258.csv',  'moth sits on flower'),
 ('\\L0.1_c-3_m5_7_47239.csv',  'moth sits on flower'),
 ('\\L0.1_c-3_m5_8_48961.csv',  'moth sits on flower'),
 ('\\L0.1_c-3_m5_10_51462.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_12_54487.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_15_58748.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_17_60830.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_18_62363.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_20_64412.csv', 'moth sits on flower'),
('\\L0.1_c-3_m5_21_65154.csv',  'removed since moth and tracking were working well'),
 ('\\L0.1_c-3_m5_22_67013.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_23_68970.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_25_70531.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_26_73715.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_28_77295.csv', 'edge case moth very briefly sits on flower 2-3 times not going to include to play it safe'),
 ('\\L0.1_c-3_m5_29_79654.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_30_81737.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_31_83229.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_32_84803.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_36_98169.csv', 'moth sits on flower'),
 ('\\L0.1_c-3_m5_37_99770.csv', 'moth sits on flower'),
 ('\\L50_c-3_m22_19_82619.csv', 'flower getting picked up'),
 ('\\L50_c-3_m35_7_49157.csv' , 'flower getting picked up'),
 ('\\L50_c-3_m51_8_39608.csv', 'abnormal moth behavior'),
('\\L0.1_c-3_m40_10_35319.csv', 'remove first 50 and last 100 frames'),
('\\L0.1_c-3_m40_12_38758.csv', 'remove first 50 and last 100 frames'),
('\\L0.1_c-3_m40_13_42800.csv',  'remove first 50 and last 100 frames'),
('\\L0.1_c-3_m40_19_71339.csv', 'remove first 50 and last 100 frames'),

 ('\\L0.1_c-3_m45_22_72759.csv', 'removed since moth sits on flower'),

 ('\\L0.1_c-3_m47_3_21452.csv', 'removed for abnormal behavior'),
 ('\\L0.1_c-3_m57_16_84843.csv', 'removed since moth sits on flower'),
 ('\\L50_c-3_m15_1_19341.csv', 'removed for abnormal behavior'),
 ('\\L50_c-3_m15_3_24522.csv', 'removed since moth sits on flower'),
('\\L50_c-3_m22_46_164536.csv', 'remove last 16 seconds * 30 frames a second = x frames'),
('\\L50_c-3_m32_6_60884.csv', 'remove last minute * 30 frames a second = x frames'),
 ('\\L50_c-3_m35_0_34946.csv', 'remove from analysis too short post shortening'),
 ('\\L50_c-3_m45_1_24243.csv', 'removed since moth sits on flower')])

In [49]:
csv_names_list = names[::,0]
csv_conditions_list = names[::,1]

In [50]:
need_to_add = [element for element in name_list if not element in csv_names_list]
conditions_to_add = ['blank']*len(need_to_add)

In [52]:
final_name_list = list(csv_names_list) + need_to_add
final_condition_list = list(csv_conditions_list) + conditions_to_add

In [53]:
names_reason = pd.DataFrame({'names': final_name_list, 'condition': final_condition_list})
names_reason.to_csv(centroids + "\\names_reason.csv")

In [7]:
names_v3 = ['L0.1_c-3_m20_1_55058',
 'L0.1_c-3_m21_28_118033',
 'L0.1_c-3_m21_33_137064',
 'L0.1_c-3_m21_39_156159',
 'L0.1_c-3_m21_44_170030',
 'L0.1_c-3_m21_9_67154',
 'L0.1_c-3_m22_0_31133',
 'L0.1_c-3_m22_1_39996',
 'L0.1_c-3_m22_3_48652',
 'L0.1_c-3_m22_4_55718',
 'L0.1_c-3_m22_6_70732',
 'L0.1_c-3_m22_7_76668',
 'L0.1_c-3_m22_8_80247',
 'L0.1_c-3_m22_9_84449',
 'L0.1_c-3_m23_7_66939',
 'L0.1_c-3_m25_0_34060',
 'L0.1_c-3_m27_3_42697',
 'L0.1_c-3_m27_6_56461',
 'L0.1_c-3_m2_0_27647',
 'L0.1_c-3_m2_3_39685',
 'L0.1_c-3_m32_11_158526',
 'L0.1_c-3_m32_6_124866',
 'L0.1_c-3_m32_7_137269',
 'L0.1_c-3_m32_9_149643',
 'L0.1_c-3_m34_10_67096',
 'L0.1_c-3_m34_12_77113',
 'L0.1_c-3_m34_13_81816',
 'L0.1_c-3_m34_2_34199',
 'L0.1_c-3_m34_4_39989',
 'L0.1_c-3_m34_5_45114',
 'L0.1_c-3_m37_1_36557',
 'L0.1_c-3_m37_2_41706',
 'L0.1_c-3_m38_1_37156',
 'L0.1_c-3_m40_0_10715',
 'L0.1_c-3_m40_14_65761',
 'L0.1_c-3_m40_15_68848',
 'L0.1_c-3_m40_19_76296',
 'L0.1_c-3_m40_20_78304',
 'L0.1_c-3_m40_3_23665',
 'L0.1_c-3_m40_7_31581',
 'L0.1_c-3_m40_9_38268',
 'L0.1_c-3_m41_0_37701',
 'L0.1_c-3_m41_6_66022',
 'L0.1_c-3_m41_7_69353',
 'L0.1_c-3_m41_8_76152',
 'L0.1_c-3_m43_0_22861',
 'L0.1_c-3_m43_7_64829',
 'L0.1_c-3_m44_6_92109',
 'L0.1_c-3_m45_0_23877',
 'L0.1_c-3_m45_21_91572',
 'L0.1_c-3_m45_24_101924',
 'L0.1_c-3_m45_4_36699',
 'L0.1_c-3_m46_10_42971',
 'L0.1_c-3_m46_11_47364',
 'L0.1_c-3_m46_1_12169',
 'L0.1_c-3_m46_9_37974',
 'L0.1_c-3_m47_15_112781',
 'L0.1_c-3_m49_0_30634',
 'L0.1_c-3_m49_11_71099',
 'L0.1_c-3_m49_16_92202',
 'L0.1_c-3_m54_11_52712',
 'L0.1_c-3_m54_17_67283',
 'L0.1_c-3_m54_18_73176',
 'L0.1_c-3_m54_19_76704',
 'L0.1_c-3_m54_1_15827',
 'L0.1_c-3_m54_20_83758',
 'L0.1_c-3_m54_21_87243',
 'L0.1_c-3_m54_22_90598',
 'L0.1_c-3_m54_25_99790',
 'L0.1_c-3_m54_26_103591',
 'L0.1_c-3_m54_28_107852',
 'L0.1_c-3_m54_30_116299',
 'L0.1_c-3_m54_31_123606',
 'L0.1_c-3_m54_33_127430',
 'L0.1_c-3_m54_34_131043',
 'L0.1_c-3_m57_12_84843',
 'L0.1_c-3_m57_13_88026',
 'L0.1_c-3_m57_21_104897',
 'L0.1_c-3_m57_30_132621',
 'L0.1_c-3_m57_32_138460',
 'L0.1_c-3_m57_40_152634',
 'L0.1_c-3_m57_8_74673',
 'L0.1_c-3_m57_9_79404',
 'L0.1_c-3_m5_0_26767',
 'L0.1_c-3_m5_1_36270',
 'L0.1_c-3_m8_1_29946',
 'L0.1_c-3_m8_2_35757',
 'L0.1_c-3_m8_5_90616',
 'L0.1_c-3_m8_6_104065',
 'L0.1_c-3_m8_7_104819',
 'L0.1_c-3_m8_8_108546',
 'L50_c-3_m10_3_49237',
 'L50_c-3_m10_4_55095',
 'L50_c-3_m12_11_68803',
 'L50_c-3_m12_4_26399',
 'L50_c-3_m12_5_36078',
 'L50_c-3_m12_9_61733',
 'L50_c-3_m13_3_49149',
 'L50_c-3_m14_0_27702',
 'L50_c-3_m15_10_82712',
 'L50_c-3_m15_11_85922',
 'L50_c-3_m15_2_23243',
 'L50_c-3_m15_3_36339',
 'L50_c-3_m15_5_52197',
 'L50_c-3_m15_8_63457',
 'L50_c-3_m21_9_102786',
 'L50_c-3_m22_0_26328',
 'L50_c-3_m22_10_90709',
 'L50_c-3_m22_11_101580',
 'L50_c-3_m22_13_110447',
 'L50_c-3_m22_14_131692',
 'L50_c-3_m22_16_142707',
 'L50_c-3_m22_18_150882',
 'L50_c-3_m22_19_155634',
 'L50_c-3_m22_1_29836',
 'L50_c-3_m22_20_159598',
 'L50_c-3_m22_22_168341',
 'L50_c-3_m22_2_34473',
 'L50_c-3_m22_3_43394',
 'L50_c-3_m22_5_63433',
 'L50_c-3_m22_6_72833',
 'L50_c-3_m22_7_78578',
 'L50_c-3_m22_8_82619',
 'L50_c-3_m22_9_89351',
 'L50_c-3_m32_5_73263',
 'L50_c-3_m35_0_34946',
 'L50_c-3_m35_1_44672',
 'L50_c-3_m35_2_49157',
 'L50_c-3_m37_1_33342',
 'L50_c-3_m39_0_33120',
 'L50_c-3_m39_13_96193',
 'L50_c-3_m39_19_126661',
 'L50_c-3_m39_1_38504',
 'L50_c-3_m39_8_62255',
 'L50_c-3_m45_0_23064',
 'L50_c-3_m6_2_35112',
 'L50_c-3_m9_1_39710']

In [12]:
csv_list = glob.glob(centroids + '\\*c-3*.csv')
csv_list

['E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m10_firstVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m10_lastVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m12_firstVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m12_lastVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m20_firstVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m20_lastVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m21_firstVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m21_lastVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m22_firstVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m22_lastVisit.csv',
 'E:\\Downloads\\ManducaMultiSenseData\\Motion\\Centroids_v3\\L0.1_c-3_m23_firstVisit

In [22]:
name

'L0.1_c-3_m20_1_55058'

In [25]:
cutoff = 55
for name in names_v3[1:]:
    
    df = pd.read_csv(centroids + "\\" + name + ".csv")
    

    
    window = pd.read_csv(window_path)
    
    if np.any(window.name.str.contains(str(name))):
        
    
        start,end = Frame(name,window)



        frameNum = np.arange(start,end)




        x,y = df.X_position.values, df.Y_position.values

        video_name = video_list +  "//PotentialOutlier/" + name  + '.mp4'

        fourcc = cv2.VideoWriter_fourcc(*'mp4v') 

        video = cv2.VideoWriter(video_name, fourcc, 30, (648,488))






        if name.startswith('L0.1'):
            name = name[:12]
            if name.endswith('_'):
                name = name[:-1]
        elif name.startswith('L50'):
            name = name[:11]
            if name.endswith('_'):
                name = name[:-1]


        print(name)



        i = 0


        while i < (len(x)-1):

    #         fig = plt.figure(figsize = (20,6))
            back, cur, diff, f = VideoPull(0,frameNum[i], name, video_list)
            c = [0,0,255]
            if np.isnan(x[i]):
                x[i], y[i] = 0,0
                c = [0,255,0]
            cv2.circle(f, (int(x[i]*74/2.5),int(y[i]*74/2.5)), 6, c, 3)

            i+=1
    #         plt.imshow(cur, cmap = 'gray')
            video.write(f)
    #         cv2.imshow('video',cur)
    #         if (cv2.waitKey(1) & 0xFF) == ord('q'): # Hit `q` to exit
    #             break

        cv2.destroyAllWindows()
        video.release()
    else:
        print("already filtered out " + str(name))

L0.1_c-3_m21
L0.1_c-3_m21
L0.1_c-3_m21
L0.1_c-3_m21
L0.1_c-3_m21
L0.1_c-3_m22
L0.1_c-3_m22
L0.1_c-3_m22
L0.1_c-3_m22
L0.1_c-3_m22
L0.1_c-3_m22
L0.1_c-3_m22
L0.1_c-3_m22
L0.1_c-3_m23
L0.1_c-3_m25
L0.1_c-3_m27
L0.1_c-3_m27
L0.1_c-3_m2
L0.1_c-3_m2
L0.1_c-3_m32
L0.1_c-3_m32
L0.1_c-3_m32
L0.1_c-3_m32
L0.1_c-3_m34
L0.1_c-3_m34
L0.1_c-3_m34
L0.1_c-3_m34
L0.1_c-3_m34
L0.1_c-3_m34
L0.1_c-3_m37
L0.1_c-3_m37
L0.1_c-3_m38
L0.1_c-3_m40
L0.1_c-3_m40
L0.1_c-3_m40
L0.1_c-3_m40
L0.1_c-3_m40
L0.1_c-3_m40
L0.1_c-3_m40
L0.1_c-3_m40
L0.1_c-3_m41
L0.1_c-3_m41
L0.1_c-3_m41
L0.1_c-3_m41
L0.1_c-3_m43
L0.1_c-3_m43
L0.1_c-3_m44
L0.1_c-3_m45
L0.1_c-3_m45
L0.1_c-3_m45
L0.1_c-3_m45
L0.1_c-3_m46
L0.1_c-3_m46
L0.1_c-3_m46
L0.1_c-3_m46
L0.1_c-3_m47
L0.1_c-3_m49
L0.1_c-3_m49
L0.1_c-3_m49
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m54
L0.1_c-3_m57
L0.1_c-3_m57
L0.1_c-3_m57
L

In [3]:
remove_list = r'E:\Downloads\ManducaMultiSenseData\Videos\PotentialOutlier\\'
name_video_file = glob.glob(remove_list + "*.mp4")

In [4]:
name_list = []
for n in name_video_file:
    name = "\\" + n[59:-4] + ".csv"
    name_list.append(name)

In [5]:
name_list

['\\L0.1_c-3_m23_6_66939.csv',
 '\\L0.1_c-3_m32_16_149643.csv',
 '\\L0.1_c-3_m32_17_154000.csv',
 '\\L0.1_c-3_m34_11_45114.csv',
 '\\L0.1_c-3_m34_14_51383.csv',
 '\\L0.1_c-3_m34_19_67675.csv',
 '\\L0.1_c-3_m34_19_68612.csv',
 '\\L0.1_c-3_m34_2_29617.csv',
 '\\L0.1_c-3_m37_0_36557.csv',
 '\\L0.1_c-3_m38_0_24374.csv',
 '\\L0.1_c-3_m39_10_40618.csv',
 '\\L0.1_c-3_m39_11_41749.csv',
 '\\L0.1_c-3_m39_12_43448.csv',
 '\\L0.1_c-3_m39_13_44002.csv',
 '\\L0.1_c-3_m39_2_31673.csv',
 '\\L0.1_c-3_m39_3_32250.csv',
 '\\L0.1_c-3_m39_4_35326.csv',
 '\\L0.1_c-3_m39_5_35876.csv',
 '\\L0.1_c-3_m39_6_37544.csv',
 '\\L0.1_c-3_m39_7_37856.csv',
 '\\L0.1_c-3_m39_8_39080.csv',
 '\\L0.1_c-3_m39_9_40013.csv',
 '\\L0.1_c-3_m45_22_72759.csv',
 '\\L0.1_c-3_m45_5_40142.csv',
 '\\L0.1_c-3_m46_13_47364.csv',
 '\\L0.1_c-3_m47_3_21452.csv',
 '\\L0.1_c-3_m47_4_22585.csv',
 '\\L0.1_c-3_m57_16_84843.csv',
 '\\L0.1_c-3_m5_0_26767.csv',
 '\\L0.1_c-3_m5_10_51462.csv',
 '\\L0.1_c-3_m5_12_54487.csv',
 '\\L0.1_c-3_m5_15_58748.

In [54]:
csv_names_list = names[::,0]
csv_conditions_list = names[::,1]

In [55]:
need_to_add = [element for element in name_list if not element in csv_names_list]
conditions_to_add = ['blank']*len(need_to_add)

In [56]:
final_name_list = list(csv_names_list) + need_to_add
final_condition_list = list(csv_conditions_list) + conditions_to_add

In [57]:
names_reason = pd.DataFrame({'names': final_name_list, 'condition': final_condition_list})
names_reason.to_csv(centroids + "\\names_reason.csv")

In [4]:
new_df = pd.read_csv(centroids + "\\names_reason.csv")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 39: invalid start byte